In [38]:
#Vectorizo normalizo, separo 80 20 modelo euclidiano
#Fecha nacimiento, tamaño, color, temperamento, nivel de actividad, peso, nivel de socialización, vacunado
from datetime import datetime, timedelta


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors


In [4]:
df = pd.read_csv('dades_mascotas.csv', encoding='utf-8')
df.head()

,id,fecha_nacimiento,tamano,color,temperamento,nivel_actividad,peso,nivel_socializacion,vacunado
0,1,2015-05-14,Mediano,Negro,Sociable,Alto,23.45,Alta,True
1,2,2018-02-23,Grande,Blanco,Tranquilo,Bajo,45.67,Media,False
2,3,2016-08-09,Pequeño,Gris,Juguetón,Medio,12.34,Alta,True
3,4,2019-11-30,Mediano,Marrón,Activo,Alto,18.50,Media,False
4,5,2014-07-22,Pequeño,Amarillo,Independiente,Medio,9.10,Baja,True


In [5]:
print(df.columns)


Index(['id', 'fecha_nacimiento', 'tamano', 'color', 'temperamento',
       'nivel_actividad', 'peso', 'nivel_socializacion', 'vacunado'],
      dtype='object')


In [ ]:
#TODO
#Convertir los campos fecha_nacimiento a un valor numérico
#Una vez lo haga, podré hacer el final

# # Convertir 'fecha_nacimiento' a tipo datetime
# df['fecha_nacimiento'] = pd.to_datetime(df['fecha_nacimiento'])

# # Obtener la fecha actual
# fecha_actual = pd.to_datetime(datetime.now())

# # Calcular la diferencia en días
# df['fecha_nacimiento'] = (fecha_actual - df['fecha_nacimiento']).dt.days

# # Ver el DataFrame resultante
# # print(df)

# df['fecha_nacimiento'] = (df['fecha_nacimiento'] - pd.to_datetime('1970-01-01')) // pd.Timedelta('1D')
# print(df['fecha_nacimiento'])

In [ ]:


columnas_categoricas = ['tamano', 'color', 'temperamento', 'nivel_actividad', 'nivel_socializacion', 'vacunado']
#columnas_numericas = ['fecha_nacimiento','peso']
columnas_numericas = ['peso']


# aqui lo que hago es crear un transformador para las columnas categóricas (one-hot eencoding)
# y para las columnas numéricas (Normalización)

preprocesador = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), columnas_categoricas),  #transformo las categorías en variables binarias
        ('num', StandardScaler(), columnas_numericas)    #y normAlizo las características numéricas
    ])

# creo el pipeline que preprocesa los datos
pipeline = Pipeline(steps=[('preprocesar', preprocesador)])

# el pipeline lo aplico para preprocesar las características
X = pipeline.fit_transform(df)

# creo un modelo KNN
# aqui lo que hago es usar NearestNeighbors para encontrar las 10 mascotas más cercanas
knn = NearestNeighbors(n_neighbors=10, metric='euclidean')

# ajusto el modelo con las características preprocesadas
knn.fit(X)

#notas: 
nueva_mascota = pd.DataFrame({
    #'fecha_nacimiento': [datetime.now() - timedelta(days=3*365)], 
    'tamano': ['Mediano'],
    'color': ['Blanco'],
    'temperamento': ['Tranquilo'],
    'nivel_actividad': ['Alto'],
    'peso': [50], 
    'nivel_socializacion': ['Alta'],  
    'vacunado': [True] 
})


# transformo la nueva mascota con el mismo pipeline
X_nueva = pipeline.transform(nueva_mascota)

# aqui enceuntro la mascota más cercana
distancias, indices = knn.kneighbors(X_nueva)

# Mostrar el ID de la mascota más cercana
# mascota_mas_cercana = df.iloc[indices[0][0]]
# print(f'La mascota más cercana es: {mascota_mas_cercana["id"]}')

print('Las mascotas más cercanas son:')
for idx in indices[0]:
    mascota_cercana = df.iloc[idx]
    print(f'ID: {mascota_cercana["id"]} - Distancia: {distancias[0][indices[0].tolist().index(idx)]}')


Las mascotas más cercanas son:
ID: 1 - Distancia: 2.725109157605319
ID: 6 - Distancia: 2.8284271247461903
ID: 2 - Distancia: 2.844491189556304
ID: 10 - Distancia: 2.9130835658712018
ID: 9 - Distancia: 3.4448849325477453
ID: 4 - Distancia: 3.580906210472877
ID: 7 - Distancia: 3.7627797812488613
ID: 8 - Distancia: 3.806909828039149
ID: 3 - Distancia: 3.8592242341633947
ID: 5 - Distancia: 4.258026662777181
